## Chapter 10: Algorithm Analysis

*Definition*: Consider two functions $f$ and $g$. We say that $f(n)$ is $O(g(n))$ or “big-O” of $g(n)$ if

$$\lim_{n \rightarrow \infty}\frac{f(n)}{g(n)} = C$$

for some $0<C<\infty$

Generally, $g(n)$ is a simple function, like a power, exponential or a power times a log.

It's a good time to know how fast functions grow relative to each other.  List the following in lowest to highest growth. $f(n)$ grows faster than $g(n)$ if $\lim_{n \rightarrow \infty} f(n)/g(n) = \infty$.
- $e^n$
- $n^2$ 
- $e^{n^2}$
- $n^2 \ln(n)$
- $n!$

In [ ]:
fig, ax = lines(0..10, n->exp(n), label = "e^n")
lines!(ax, 0..10, n->n^2, label = "n^2")
axislegend(ax, position = :lt)
fig

Find big-O for the following:
* $n^2+e^n$
* $3n^2+e^{-n}$
* $n + n \ln (n)$

### Polynomial Evaluation

Recall that a polynomial is

$$p(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots + a_n x^n$$

For example, a quadratic is a polynomial like $q(x)= 3x^2+7x-2$

The simple way to evaluate the polynomial is to find each power of x and then multiply by the coefficient.  Here's a function that takes an array of coefficients and a value $x$ and calculates $p(x)$.  

In [ ]:
function polyEval(coeffs::Vector{T}, x::S) where {T <: Number, S <: Number}
  local sum = zero(T)

  # we will use a simple power function that is not very efficient:
  function pow(x::T,n::Int) where T <: Number
    local prod = one(T)
    for j=1:n
      prod *=x
    end
    prod
  end

  # then we have the sum of all of the terms:
  for n=1:length(coeffs)
    sum += coeffs[n]*pow(x,n-1)
  end
  sum
end

note: we will be explaining the `where {T <: Number, S <: Number}` later.  This allows the ability to evaluate polynomials at any type of number. 

The following evaluates $q(1.5)$ for the quadratic above

In [ ]:
polyEval([-2,7,3],1.5)

In [ ]:
-2+7*(1.5)+3*(1.5)^2

And here is the polynomial $p(x)=1+2x+3x^2+4x^3$ evaluated at $x=4$

In [ ]:
polyEval([1,2,3,4],4)

We want to see how polynomial evaluation occurs for a given value of $n$, the degree of the polynomial.  We can do this two different ways:

1. Use some analytic techniques
2. Run some code and analyze.

For #1, we note that the number of multiplications for evaluating a polynomial of degree $n$ is

$$0+1+2+3+4+\cdots+(n+1)= \frac{(n+1)(n+2)}{2}$$

The number of additions is $n$. Overall the number of operations then is

$$\frac{(n+1)(n+2)}{2} +n$$

What is the order (big-o) of this?

For #2, we will do the following.  First need to load (and probably add) some packages

In [ ]:
using BenchmarkTools, CairoMakie, LsqFit, Random
CairoMakie.activate!()
Makie.inline!(true)

In [ ]:
Random.seed!(132)

The following make an array of times that store the evaluate times of the polynomial evaluation.  Although we only fill part of the array so it doesn't take foreever.

In [10]:
times = zeros(351)
r = 1:50:351
for i=r
  coeffs = rand(i+1)
  times[i] = @belapsed polyEval($coeffs,1/3)
end

In [ ]:
fig, ax = scatter(r,times[r])

Although this looks quadratic, let's see if it is.  We can find the data to a quadratic using the following:

In [ ]:
 model(t, p) = p[1].+p[2].*t.+p[3].*t.^2

In [14]:
fit = curve_fit(model, r, times[r], [1e-8,1e-8,1e-8]);

And if we want the parameters

In [ ]:
fit.param

The following is the confidence intervals for each of the paramters. 

In [ ]:
confidence_interval(fit, 0.05)

Because the first two contain 0, we don't include them and remodel using only the $t^2$ term: 

In [17]:
fit2 = curve_fit((t,p)->p[1].*t.^2,r,times[r],[1e-8]);

In [ ]:
lines!(ax, 0..375,n->fit2.param[1]*n^2)
fig

### Horner's Method
Next, we are going to do an alternative way of evaluating a polynomial.  This is called Horner's form. 

The polynomial

$$p(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots + a_n x^n$$

can be written:

$$(a_n x +a_{n-1})x + a_{n-2})x + \cdots + a_2)x^2 + a_1)x + a_0$$ 

and the result is that there are $n$ multiplications and $n$ additions. So this should be faster.

In [ ]:
function horner(coeffs::Vector{T},x::S) where {T <: Number, S <: Number}
  result = coeffs[end]
  for i=length(coeffs)-1:-1:1
    result = x*result+coeffs[i]
  end
  result
end

In [24]:
htimes = zeros(Float64,351)
for i=r
  coeffs = rand(Float64,i)
  htimes[i] = @belapsed horner($coeffs,1/3)
end

In [ ]:
fig, ax = scatter(r,htimes[r])

In [26]:
fit3 = curve_fit(model, r, htimes[r], [1e-8,1e-8,1e-8]);

In [ ]:
fit3.param

In [ ]:
confidence_interval(fit3)

Only the 2nd parameter (the linear one) is significant so generate a new model with just that term:

In [29]:
fit4 = curve_fit((t,p)->p[1].*t,r,htimes[r],[1e-8]);

In [ ]:
lines!(ax,0..375, n->fit4.param[1]*n)
fig

In [ ]:
fig, ax = scatter(r,times[r])
scatter!(ax, r,htimes[r])
fig

### Testing the speed of Primes

From Chapter 9, we developed the following function that tests for being prime.  In this section, we will determine the order of this algorithm:

In [ ]:
function isPrime5(n::Integer)
  if n%2==0
    return true
  end
  for k=3:2:round(Int,sqrt(n))
    if n%k==0
      return false
    end
  end
  true
end

In [37]:
using Primes

The following generates an array of prime numbers and then determines the elasped time  to determine whether or not each is a prime number.  Note: we are only going to check primes, because these are the worst-case. The function will bail out early (less time) if the number is not prime. 

In [ ]:
the_primes = map(nextprime,5_000_000:5_000_000:100_000_000)

In [39]:
prime_times=Float64[]
for p in the_primes
  push!(prime_times,@belapsed isPrime5($p))
end

In [ ]:
fig, ax = scatter(the_primes,prime_times)

Let's build a model with just a constant term, a square root and a log term based on the plot:

In [41]:
fit5 = curve_fit((t, p) -> p[1].+p[2].*sqrt.(t).+p[3]*log.(t),the_primes, prime_times, [1e-4, 1e-4,1e-4]);

In [ ]:
fit5.param

In [ ]:
confidence_interval(fit5)

Only the $\sqrt{n}$ term is significant.  Let's fit the model to just that one:

In [44]:
fit6 = curve_fit((t,p) -> p[1].*sqrt.(t),the_primes,prime_times,[1e-6]);

In [ ]:
fit6.param

In [ ]:
lines!(ax,1..100_000_000,t->fit6.param[1]*sqrt(t))
fig

This shows using some data that the order of operations (big-O) of finding prime numbers using this algorithm is $O(\sqrt{n})$